In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
# import dagshub
# dagshub.init(repo_owner='y.sharma.nagarajarao', repo_name='Kidney-Disease-Classification', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

ModuleNotFoundError: No module named 'dagshub'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/y.sharma.nagarajarao/Kidney-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "y.sharma.nagarajarao"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c82315a0bbbc40a16b087fbee586c1099ca58004"
# python script.py


In [6]:
os.getcwd()

'c:\\Users\\yashp\\Documents\\Kidney-Disease-Classification'

In [8]:
import tensorflow as tf
from pathlib import Path

model = tf.keras.models.load_model(Path('artifacts\\training\\model.h5'))

In [9]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
# ConfigurationManager

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = Path("artifacts\\training\\model.h5"),
            training_data = Path("artifacts\\data_ingestion\\kidney-ct-scan-image"),
            all_params = self.params,
            mlflow_uri = "https://dagshub.com/y.sharma.nagarajarao/Kidney-Disease-Classification.mlflow",
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

        

In [14]:
# component
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [16]:
# Pipeline

try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-19 23:15:09,029: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-19 23:15:09,056: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-19 23:15:09,061: INFO: common: created directory at: artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_evaluation_config'